In [1]:
import random
import numpy as np

# Générer le graphe TSP incomplet
def generate_tsp_incomplete(num_sommets, max_poids_arete):
    matrice = [[None for _ in range(num_sommets)] for _ in range(num_sommets)]
    intervalles = {}
    for i in range(num_sommets):
        intervalle_start = random.randint(0, 10)
        intervalle_end = intervalle_start + random.randint(5, 20)
        intervalles[i] = (intervalle_start, intervalle_end)
    for i in range(num_sommets):
        for j in range(i + 1, num_sommets):
            if random.choice([True, False]):
                poids = random.randint(1, max_poids_arete)
                matrice[i][j] = poids
                matrice[j][i] = poids
    for i in range(num_sommets):
        pas_connexe = True
        for j in range(num_sommets):
            if matrice[i][j] is None:
                matrice[i][j] = float('inf')
            else:
                pas_connexe = False
        if pas_connexe:
            poids = random.randint(1, max_poids_arete)
            matrice[i][num_sommets-1] = poids
            matrice[num_sommets-1][i] = poids
    return np.array(matrice), intervalles

num_sommets = 5
max_poids_arete = 10 

# Générer les données du problème
tsp_matrice, tsp_intervalles = generate_tsp_incomplete(num_sommets, max_poids_arete)

# Afficher la matrice pondérée
print("\nMatrice pondérée:")
for row in tsp_matrice:
    print(row)

# Afficher les intervalles
print("\nIntervalles des sommets")
print(tsp_intervalles)  


Matrice pondérée:
[inf inf inf  5.  4.]
[inf inf  8. inf inf]
[inf  8. inf  1.  1.]
[ 5. inf  1. inf  1.]
[ 4. inf  1.  1. inf]

Intervalles des sommets
{0: (1, 7), 1: (10, 28), 2: (5, 23), 3: (3, 22), 4: (9, 17)}


In [2]:
import random
import numpy as np 

# Calculer la longueur d'un chemin donné
def calculate_path_length(matrice, path):
    length = 0
    for i in range(len(path) - 1):
        length += matrice[path[i]][path[i + 1]]
    length += matrice[path[-1]][path[0]]  # Retourner au point de départ
    return length

# Implémentation de l'algorithme hill climbing
def hill_climbing(matrice, num_iterations=1000):
    num_sommets = len(matrice) 
    
    # Générer une solution initiale aléatoire
    current_solution = list(range(num_sommets))
    random.shuffle(current_solution)
    current_length = calculate_path_length(matrice, current_solution)
    
    for _ in range(num_iterations):
        # Générer une solution voisine en échangeant deux sommets
        next_solution = current_solution[:]
        i, j = random.sample(range(num_sommets), 2)
        next_solution[i], next_solution[j] = next_solution[j], next_solution[i]
        
        next_length = calculate_path_length(matrice, next_solution)
        
        # Si la solution voisine est meilleure, l'adopter
        if next_length < current_length:
            current_solution = next_solution
            current_length = next_length
            
    return current_solution, current_length 

In [6]:
# best_path, best_path_length = hill_climbing()   
# print(f"Meilleur chemin trouvé pour eme iteration : {best_path} avec une longueur de {best_path_length}")   

In [7]:
tsp_matrice, tsp_intervalles = generate_tsp_incomplete(num_sommets, max_poids_arete)

# Afficher la matrice pondérée
print("\nMatrice pondérée:")
for row in tsp_matrice:
    print(row)

# Afficher les intervalles
print("\nIntervalles des sommets:")
print(tsp_intervalles)

# Résoudre le problème avec l'algorithme hill climbing
solution, length = hill_climbing(tsp_matrice)

# Afficher la solution trouvée et sa longueur
print("\nSolution trouvée par hill climbing:")
print("Chemin:", solution)
print("Longueur du chemin:", length)  


Matrice pondérée:
[inf inf  9. inf inf]
[inf inf inf  8.  1.]
[ 9. inf inf  7.  2.]
[inf  8.  7. inf  6.]
[inf  1.  2.  6. inf]

Intervalles des sommets:
{0: (5, 25), 1: (1, 17), 2: (2, 13), 3: (2, 18), 4: (6, 13)}

Solution trouvée par hill climbing:
Chemin: [0, 4, 1, 2, 3]
Longueur du chemin: inf
